# 📦 TheLook E-commerce Return Data Preparation

This notebook will:

1. **Load** the four raw CSV files  
   - `distribution_centers`  
   - `order_items`  
   - `products`  
   - `users`  
2. **Clean** the data and add key features  
3. **Select** the columns needed for modelling  
4. **Save** the prepared set as `thelook_returns_features.csv` (`RETURN_FLAG 1 = returned, 0 = kept`) in the current folder

## 1. **Load** the four raw CSV files 

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams['figure.dpi'] = 110  # sharper charts

# Edit these paths if your CSVs live elsewhere
path_dc  = Path('../../1_datasets/thelook_Ecommerce/distribution_centers.csv')
path_ord = Path('../../1_datasets/thelook_Ecommerce/order_items.csv')
path_prod = Path('../../1_datasets/thelook_Ecommerce/products.csv')
path_users = Path('../../1_datasets/thelook_Ecommerce/users.csv')

# ---------- Load ----------
dc   = pd.read_csv(path_dc)
orders  = pd.read_csv(path_ord,  parse_dates=['created_at','shipped_at','delivered_at','returned_at'])
products= pd.read_csv(path_prod)
users   = pd.read_csv(path_users, parse_dates=['created_at'])

print('Shapes:')
for name, df in [('DistributionCenters', dc),
                 ('OrderItems', orders),
                 ('Products', products),
                 ('Users', users)]:
    print(f'  {name:<18} → {df.shape}')


Shapes:
  DistributionCenters → (10, 4)
  OrderItems         → (180952, 11)
  Products           → (29120, 9)
  Users              → (100000, 15)


## **Preview**  (4 rows each)

In [2]:
print("\n📄 Preview (4 rows each) -----------------")
for name, df in [("distribution_centers", dc),
                 ("order_items",          orders),
                 ("products",             products),
                 ("users",                users)]:
    print(f"\n== {name} ==")
    display(df.head(4))          # Jupyter; replace with print(df.head(4)) if running as .py



📄 Preview (4 rows each) -----------------

== distribution_centers ==


,id,name,latitude,longitude
0,1,Memphis TN,35.1174,-89.9711
1,2,Chicago IL,41.8369,-87.6847
2,3,Houston TX,29.7604,-95.3698
3,4,Los Angeles CA,34.0500,-118.2500



== order_items ==


,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,15721,10826,8614,13606,42278,Shipped,2020-06-28 00:40:17 UTC,2020-06-29 04:40:00 UTC,NaN,NaN,2.5
1,19167,13243,10505,13606,51560,Shipped,2022-03-01 05:18:44 UTC,2022-03-01 20:48:00 UTC,NaN,NaN,2.5
2,77007,53140,42340,13606,207367,Shipped,2021-04-11 01:31:52 UTC,2021-04-14 00:32:00 UTC,NaN,NaN,2.5
3,151639,104681,83704,13606,408715,Shipped,2022-03-29 21:37:06 UTC,2022-03-30 12:41:00 UTC,NaN,NaN,2.5



== products ==


,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,27569,92.652563,Swim,2XU Men's Swimmers Compression Long Sleeve Top,2XU,150.410004,Men,B23C5765E165D83AA924FA8F13C05F25,1
1,27445,24.719661,Swim,TYR Sport Men's Square Leg Short Swim Suit,TYR,38.990002,Men,2AB7D3B23574C3DEA2BD278AFD0939AB,1
2,27457,15.897600,Swim,TYR Sport Men's Solid Durafast Jammer Swim Suit,TYR,27.600000,Men,8F831227B0EB6C6D09A0555531365933,1
3,27466,17.850000,Swim,TYR Sport Men's Swim Short/Resistance Short Sw...,TYR,30.000000,Men,67317D6DCC4CB778AEB9219565F5456B,1



== users ==


,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,44262,Michael,Sanchez,michaelsanchez@example.net,48,M,Mie,5379 Kim Corner,513-0836,Suzuka City,Japan,34.851814,136.508713,Facebook,2020-12-05 14:39:00 UTC
1,61852,David,Watson,davidwatson@example.org,21,M,Mie,58568 Brooks Plain Apt. 269,513-0836,Suzuka City,Japan,34.851814,136.508713,Search,2022-01-24 13:00:00 UTC
2,82418,Lisa,Rivera,lisarivera@example.net,29,F,Mie,3092 Perez Overpass,513-0836,Suzuka City,Japan,34.851814,136.508713,Search,2019-09-07 07:29:00 UTC
3,23274,Logan,Flores,loganflores@example.com,53,M,Acre,412 Underwood Tunnel Suite 025,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2020-06-28 14:39:00 UTC


## 2. **Clean** the data and add key features 

In [6]:
# -------------------------------------------------
# 2. Minimal cleaning / feature engineering
# -------------------------------------------------
orders["RETURN_FLAG"] = orders["returned_at"].notna().astype(int)
orders["basket_size"] = orders.groupby("order_id")["id"].transform("count")

# ------- Merge tables (keep DC name instead of raw ID) -------
df = (
    orders
    .merge(
        products[[
            "id", "category", "department", "brand",
            "retail_price", "cost", "distribution_center_id"
        ]],
        left_on="product_id", right_on="id", how="left", suffixes=("", "_prod")
    )
    .merge(
        users[["id", "gender", "age", "country", "created_at"]],
        left_on="user_id", right_on="id", how="left", suffixes=("", "_user")
    )
    # merge on numeric ID, then bring the DC **name**
    .merge(
        dc.rename(columns={"id": "distribution_center_id", "name": "distribution_center_name"}),
        on="distribution_center_id", how="left"
    )
)

# Rename for clarity
df = df.rename(columns={
    "created_at": "order_created_at",          # from orders
    "created_at_user": "user_created_at"       # from users
})

# Datetime & numeric coercion
for col in ["order_created_at", "user_created_at", "shipped_at"]:
    df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)

for col in ["sale_price", "retail_price", "cost"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Derived columns
df["discount_pct"] = (df["retail_price"] - df["sale_price"]) / df["retail_price"]
df["discount_pct"] = df["discount_pct"].clip(0, 1)

df["tenure_days"]       = (df["order_created_at"] - df["user_created_at"]).dt.days
df["ship_latency_days"] = (df["shipped_at"]      - df["order_created_at"]).dt.days

df["created_hour"]      = df["order_created_at"].dt.hour
df["created_dayofweek"] = df["order_created_at"].dt.day_name()
df["created_month"]     = df["order_created_at"].dt.month

SEASON = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Fall", 10: "Fall", 11: "Fall"
}
df["season"] = df["created_month"].map(SEASON)

# df now has a readable `distribution_center_name` column alongside the numeric ID.

## 3. **Select** the columns needed for modelling 

In [7]:
# -------------------------------------------------
# 3. **Select** the columns needed for modelling 
# -------------------------------------------------
cols = ["gender","age","country","tenure_days",
        "category","department","brand",
        "retail_price","sale_price","discount_pct","cost",
        "distribution_center_name","basket_size",
        "ship_latency_days","created_hour","created_dayofweek","season",
        "RETURN_FLAG"]

feature_df = df[cols].dropna(subset=["RETURN_FLAG"])


## 4. **Save** the prepared dataset

In [8]:
from pathlib import Path

# ---------------------------------------------
# Save prepared dataset in the current directory
# ---------------------------------------------
out_path = Path("thelook_returns_features.csv")
feature_df.to_csv(out_path, index=False)

print(
    f"✅ Saved {feature_df.shape[0]} rows × {feature_df.shape[1]} columns →",
    out_path.resolve()
)

# optional peek
feature_df.head()

✅ Saved 180952 rows × 18 columns → /Users/pyaelinn/Desktop/CDSP/ET6-CDSP-group-17-repo-Mission_Impostible/2_data_preparation/theLook_Ecommerce/thelook_returns_features.csv


,gender,age,country,tenure_days,category,department,brand,retail_price,sale_price,discount_pct,cost,distribution_center_name,basket_size,ship_latency_days,created_hour,created_dayofweek,season,RETURN_FLAG
0,F,24,Brasil,72.0,Accessories,Women,Scarf_tradinginc,2.5,2.5,0.0,1.1075,Houston TX,1,1.0,0.0,Sunday,Summer,0
1,F,23,Brasil,1151.0,Accessories,Women,Scarf_tradinginc,2.5,2.5,0.0,1.1075,Houston TX,1,0.0,5.0,Tuesday,Spring,0
2,F,59,China,650.0,Accessories,Women,Scarf_tradinginc,2.5,2.5,0.0,1.1075,Houston TX,1,2.0,1.0,Sunday,Spring,0
3,F,41,United States,757.0,Accessories,Women,Scarf_tradinginc,2.5,2.5,0.0,1.1075,Houston TX,1,0.0,21.0,Tuesday,Spring,0
4,F,18,China,194.0,Accessories,Women,Scarf_tradinginc,2.5,2.5,0.0,1.1075,Houston TX,1,2.0,9.0,Wednesday,Winter,0
